In [254]:
import numpy as np
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt 

In [48]:
trainset = MNIST(root='Desktop', train=True, download=True)
testset  = MNIST(root='Desktop', train=False, download=True)

trainset_array = trainset.data.numpy()
testset_array  = testset.data.numpy()
y_train_array = trainset.targets.numpy()
y_test_array  = testset.targets.numpy()

In [53]:
trainset_array.shape

(60000, 28, 28)

In [243]:
class Conv_layer:

    def __init__(self ,filter_size ,num_filters):
        """
        """
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = np.random.randn(num_filters, self.filter_size, self.filter_size) / self.filter_size**2
    
    def image_region(self, image):
        
        h, w = image.shape
        self.image = image
        for i in range(h - self.filter_size + 1):
            for j in range(w - self.filter_size + 1):
                image_patch = image[i : (i + self.filter_size), j : (j + self.filter_size)]
                yield image_patch, i, j
        
    def forward(self, image):
        
        h, w = image.shape
        conv_output = np.zeros((h - self.filter_size + 1, w - self.filter_size + 1, self.num_filters))
        for image_patch, i, j in self.image_region(image): 
            conv_output[i, j] = np.sum(image_patch*self.filters , axis=(1,2))
        return conv_output

    def backprop(self, dL_dout, lr):
        
        dL_dFilter = np.zeros(self.filters.shape)
        for image_patch, i, j in self.image_region(self.image):
            for k in range(self.num_filters):
                dL_dFilter[k] += image_patch * dL_dout[i, j, k]
        self.filters -= lr*dL_dFilter
        return dL_dFilter
        

In [244]:
class Maxpool :
    
    def __init__(self, filter_size):
        
        self.filter_size = filter_size
    
    def image_region(self, image):
        
        self.image = image
        h = image.shape[0] // self.filter_size
        w = image.shape[1] // self.filter_size
        
        for i in range(h):
            for j in range(w):
                image_patch = image[(i * self.filter_size) : (i * self.filter_size + self.filter_size), 
                                            (j * self.filter_size) : (j * self.filter_size + self.filter_size)]
                yield image_patch, i, j
    
    def forward(self, image):
        
        h, w, num_filters = image.shape
        maxpool_output = np.zeros((h // self.filter_size, w // self.filter_size, num_filters))
        
        for image_patch, i, j in self.image_region(image):
            maxpool_output[i, j] = np.amax(image_patch, axis = (0,1))
        return maxpool_output
    
    def backprop(self, dL_dout):
        
        dL_dMaxpool = np.zeros(self.image.shape)
        for image_patch, i, j in self.image_region(self.image):
            
            h, w, num_filters = image_patch.shape
            max_val = np.amax(image_patch, axis = (0,1))
            
            for ii in range(h):
                for jj in range(w):
                    for kk in range(num_filters):
                        if image_patch[ii, jj, kk] == max_val[kk]:
                            dL_dMaxpool[(i * self.filter_size + ii), (j * self.filter_size + jj), kk] = dL_dout[i, j, kk]
        
        return dL_dMaxpool
        

In [245]:
class Softmax:
    
    # fully connected layer with Softmax activation
    
    def __init__(self, input_node, softmax_node):
        
        self.weights = np.random.randn(input_node, softmax_node)/input_node
        self.biaises = np.random.randn(softmax_node)
        
    def forward(self, image): 
        
        self.orig_img_shape = image.shape
        image_flat          = image.flatten()
        self.flat_input     = image_flat
        output_val          = np.dot(image_flat,self.weights) + self.biaises
        self.out            = output_val
        exp_output          = np.exp(output_val)
        return exp_output / np.sum(exp_output, axis =0)
    
    def backprop(self, dL_dout, lr):
        
        for i, grad in enumerate(dL_dout):
            if grad == 0:
                continue
            
            exp_z      = np.exp(self.out)
            S          = np.sum(exp_z)
            
            dout_dz    = - exp_z[i] * exp_z / S**2
            dout_dz[i] = exp_z[i] * (S - exp_z[i]) / S**2
            
            dz_dw      = self.flat_input
            dz_db      = 1
            dz_dinp    = self.weights
            
            dL_dz      = grad * dout_dz
            dL_dw      = dz_dw[np.newaxis].T @ dL_dz[np.newaxis]
            dL_db      = dL_dz * dz_db
            dL_dinp    = dz_dinp @ dL_dz
            
            self.weights -= lr * dL_dw
            self.biaises -= lr * dL_db
            
            return dL_dinp.reshape(self.orig_img_shape)
        
            
 

In [249]:
%%time
conv = Conv_layer(8, 3)
pool = Maxpool(2)
softmax = Softmax(10 * 10 * 3, 10)
def cnn_forward(images, labels):
    
    output  = conv.forward((images / 255) - 0.5)
    output  = pool.forward(output)
    output  = softmax.forward(output)
    
    cross_ent = - np.log(output[labels])
    accuracy  = 1 if np.argmax(output) == labels else 0
    
    return output, cross_ent, accuracy

def train_cnn(images, labels, lr):
    
    output, loss, acc = cnn_forward(images, labels)
    
    gradient = np.zeros(10)
    gradient[labels] = -1 / output[labels]
    
    gradback = softmax.backprop(gradient, lr)
    gradback = pool.backprop(gradback)
    gradback = conv.backprop(gradback, lr)
    
    return loss, acc

print('MNIST CNN initialized!')

# Train the CNN for 5 epochs
for epoch in range(3):
    print('--- Epoch %d ---' % (epoch + 1))

  # Shuffle the training data
    permutation = np.random.permutation(len(trainset_array))
    train_images = trainset_array[permutation]
    train_labels = y_train_array[permutation]

  # Train!
    loss = 0
    num_correct = 0
    for i, (im, label) in enumerate(zip(train_images, train_labels)):
        if i > 0 and i % 100 == 99:
            print(
            '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
                (i + 1, loss / 100, num_correct)
                  )
            loss = 0
            num_correct = 0

        l, acc = train_cnn(im, label, 0.005)
        loss += l
        num_correct += acc

# Test the CNN
print('\n--- Testing the CNN ---')
loss = 0
num_correct = 0

for im, label in zip(testset_array, y_test_array):
    _, l, acc = cnn_forward(im, label)
    loss += l
    num_correct += acc

num_tests = len(y_test_array)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)

MNIST CNN initialized!
--- Epoch 1 ---
[Step 100] Past 100 steps: Average Loss 2.661 | Accuracy: 13%
[Step 200] Past 100 steps: Average Loss 2.130 | Accuracy: 24%
[Step 300] Past 100 steps: Average Loss 1.914 | Accuracy: 32%
[Step 400] Past 100 steps: Average Loss 1.575 | Accuracy: 46%
[Step 500] Past 100 steps: Average Loss 1.127 | Accuracy: 71%
[Step 600] Past 100 steps: Average Loss 0.844 | Accuracy: 73%
[Step 700] Past 100 steps: Average Loss 0.638 | Accuracy: 84%
[Step 800] Past 100 steps: Average Loss 0.944 | Accuracy: 71%
[Step 900] Past 100 steps: Average Loss 0.650 | Accuracy: 80%
[Step 1000] Past 100 steps: Average Loss 0.502 | Accuracy: 91%
[Step 1100] Past 100 steps: Average Loss 0.651 | Accuracy: 75%
[Step 1200] Past 100 steps: Average Loss 0.548 | Accuracy: 82%
[Step 1300] Past 100 steps: Average Loss 0.620 | Accuracy: 77%
[Step 1400] Past 100 steps: Average Loss 0.593 | Accuracy: 78%
[Step 1500] Past 100 steps: Average Loss 0.520 | Accuracy: 83%
[Step 1600] Past 100 step

[Step 13100] Past 100 steps: Average Loss 0.216 | Accuracy: 91%
[Step 13200] Past 100 steps: Average Loss 0.189 | Accuracy: 94%
[Step 13300] Past 100 steps: Average Loss 0.238 | Accuracy: 91%
[Step 13400] Past 100 steps: Average Loss 0.170 | Accuracy: 95%
[Step 13500] Past 100 steps: Average Loss 0.164 | Accuracy: 96%
[Step 13600] Past 100 steps: Average Loss 0.466 | Accuracy: 91%
[Step 13700] Past 100 steps: Average Loss 0.296 | Accuracy: 88%
[Step 13800] Past 100 steps: Average Loss 0.201 | Accuracy: 96%
[Step 13900] Past 100 steps: Average Loss 0.229 | Accuracy: 91%
[Step 14000] Past 100 steps: Average Loss 0.323 | Accuracy: 84%
[Step 14100] Past 100 steps: Average Loss 0.155 | Accuracy: 94%
[Step 14200] Past 100 steps: Average Loss 0.309 | Accuracy: 95%
[Step 14300] Past 100 steps: Average Loss 0.282 | Accuracy: 90%
[Step 14400] Past 100 steps: Average Loss 0.285 | Accuracy: 94%
[Step 14500] Past 100 steps: Average Loss 0.470 | Accuracy: 88%
[Step 14600] Past 100 steps: Average Los

[Step 26000] Past 100 steps: Average Loss 0.221 | Accuracy: 92%
[Step 26100] Past 100 steps: Average Loss 0.225 | Accuracy: 93%
[Step 26200] Past 100 steps: Average Loss 0.264 | Accuracy: 92%
[Step 26300] Past 100 steps: Average Loss 0.388 | Accuracy: 89%
[Step 26400] Past 100 steps: Average Loss 0.205 | Accuracy: 95%
[Step 26500] Past 100 steps: Average Loss 0.365 | Accuracy: 89%
[Step 26600] Past 100 steps: Average Loss 0.403 | Accuracy: 89%
[Step 26700] Past 100 steps: Average Loss 0.246 | Accuracy: 91%
[Step 26800] Past 100 steps: Average Loss 0.222 | Accuracy: 93%
[Step 26900] Past 100 steps: Average Loss 0.292 | Accuracy: 92%
[Step 27000] Past 100 steps: Average Loss 0.315 | Accuracy: 92%
[Step 27100] Past 100 steps: Average Loss 0.179 | Accuracy: 93%
[Step 27200] Past 100 steps: Average Loss 0.135 | Accuracy: 95%
[Step 27300] Past 100 steps: Average Loss 0.187 | Accuracy: 95%
[Step 27400] Past 100 steps: Average Loss 0.313 | Accuracy: 92%
[Step 27500] Past 100 steps: Average Los

[Step 38900] Past 100 steps: Average Loss 0.134 | Accuracy: 97%
[Step 39000] Past 100 steps: Average Loss 0.245 | Accuracy: 92%
[Step 39100] Past 100 steps: Average Loss 0.220 | Accuracy: 95%
[Step 39200] Past 100 steps: Average Loss 0.318 | Accuracy: 90%
[Step 39300] Past 100 steps: Average Loss 0.156 | Accuracy: 94%
[Step 39400] Past 100 steps: Average Loss 0.157 | Accuracy: 96%
[Step 39500] Past 100 steps: Average Loss 0.111 | Accuracy: 98%
[Step 39600] Past 100 steps: Average Loss 0.342 | Accuracy: 90%
[Step 39700] Past 100 steps: Average Loss 0.246 | Accuracy: 92%
[Step 39800] Past 100 steps: Average Loss 0.061 | Accuracy: 98%
[Step 39900] Past 100 steps: Average Loss 0.251 | Accuracy: 91%
[Step 40000] Past 100 steps: Average Loss 0.154 | Accuracy: 95%
[Step 40100] Past 100 steps: Average Loss 0.098 | Accuracy: 96%
[Step 40200] Past 100 steps: Average Loss 0.254 | Accuracy: 94%
[Step 40300] Past 100 steps: Average Loss 0.165 | Accuracy: 92%
[Step 40400] Past 100 steps: Average Los

[Step 51800] Past 100 steps: Average Loss 0.175 | Accuracy: 94%
[Step 51900] Past 100 steps: Average Loss 0.107 | Accuracy: 98%
[Step 52000] Past 100 steps: Average Loss 0.246 | Accuracy: 91%
[Step 52100] Past 100 steps: Average Loss 0.150 | Accuracy: 95%
[Step 52200] Past 100 steps: Average Loss 0.128 | Accuracy: 98%
[Step 52300] Past 100 steps: Average Loss 0.115 | Accuracy: 97%
[Step 52400] Past 100 steps: Average Loss 0.185 | Accuracy: 94%
[Step 52500] Past 100 steps: Average Loss 0.204 | Accuracy: 92%
[Step 52600] Past 100 steps: Average Loss 0.118 | Accuracy: 97%
[Step 52700] Past 100 steps: Average Loss 0.168 | Accuracy: 92%
[Step 52800] Past 100 steps: Average Loss 0.145 | Accuracy: 94%
[Step 52900] Past 100 steps: Average Loss 0.208 | Accuracy: 94%
[Step 53000] Past 100 steps: Average Loss 0.092 | Accuracy: 97%
[Step 53100] Past 100 steps: Average Loss 0.265 | Accuracy: 89%
[Step 53200] Past 100 steps: Average Loss 0.130 | Accuracy: 94%
[Step 53300] Past 100 steps: Average Los

...

[Step 56100] Past 100 steps: Average Loss 0.282 | Accuracy: 94%
[Step 56200] Past 100 steps: Average Loss 0.258 | Accuracy: 92%
[Step 56300] Past 100 steps: Average Loss 0.239 | Accuracy: 95%
[Step 56400] Past 100 steps: Average Loss 0.037 | Accuracy: 99%
[Step 56500] Past 100 steps: Average Loss 0.197 | Accuracy: 94%
[Step 56600] Past 100 steps: Average Loss 0.131 | Accuracy: 96%
[Step 56700] Past 100 steps: Average Loss 0.026 | Accuracy: 99%
[Step 56800] Past 100 steps: Average Loss 0.068 | Accuracy: 96%
[Step 56900] Past 100 steps: Average Loss 0.145 | Accuracy: 96%
[Step 57000] Past 100 steps: Average Loss 0.055 | Accuracy: 97%
[Step 57100] Past 100 steps: Average Loss 0.058 | Accuracy: 97%
[Step 57200] Past 100 steps: Average Loss 0.185 | Accuracy: 91%
[Step 57300] Past 100 steps: Average Loss 0.392 | Accuracy: 91%
[Step 57400] Past 100 steps: Average Loss 0.186 | Accuracy: 95%
[Step 57500] Past 100 steps: Average Loss 0.018 | Accuracy: 100%
[Step 57600] Past 100 steps: Average Lo

[Step 9100] Past 100 steps: Average Loss 0.218 | Accuracy: 95%
[Step 9200] Past 100 steps: Average Loss 0.356 | Accuracy: 91%
[Step 9300] Past 100 steps: Average Loss 0.309 | Accuracy: 90%
[Step 9400] Past 100 steps: Average Loss 0.177 | Accuracy: 97%
[Step 9500] Past 100 steps: Average Loss 0.064 | Accuracy: 98%
[Step 9600] Past 100 steps: Average Loss 0.185 | Accuracy: 93%
[Step 9700] Past 100 steps: Average Loss 0.145 | Accuracy: 96%
[Step 9800] Past 100 steps: Average Loss 0.078 | Accuracy: 97%
[Step 9900] Past 100 steps: Average Loss 0.080 | Accuracy: 98%
[Step 10000] Past 100 steps: Average Loss 0.190 | Accuracy: 94%
[Step 10100] Past 100 steps: Average Loss 0.174 | Accuracy: 95%
[Step 10200] Past 100 steps: Average Loss 0.284 | Accuracy: 97%
[Step 10300] Past 100 steps: Average Loss 0.278 | Accuracy: 95%
[Step 10400] Past 100 steps: Average Loss 0.112 | Accuracy: 94%
[Step 10500] Past 100 steps: Average Loss 0.035 | Accuracy: 99%
[Step 10600] Past 100 steps: Average Loss 0.204 |

[Step 22000] Past 100 steps: Average Loss 0.195 | Accuracy: 94%
[Step 22100] Past 100 steps: Average Loss 0.113 | Accuracy: 99%
[Step 22200] Past 100 steps: Average Loss 0.072 | Accuracy: 98%
[Step 22300] Past 100 steps: Average Loss 0.121 | Accuracy: 97%
[Step 22400] Past 100 steps: Average Loss 0.357 | Accuracy: 95%
[Step 22500] Past 100 steps: Average Loss 0.126 | Accuracy: 95%
[Step 22600] Past 100 steps: Average Loss 0.186 | Accuracy: 98%
[Step 22700] Past 100 steps: Average Loss 0.243 | Accuracy: 97%
[Step 22800] Past 100 steps: Average Loss 0.087 | Accuracy: 98%
[Step 22900] Past 100 steps: Average Loss 0.097 | Accuracy: 98%
[Step 23000] Past 100 steps: Average Loss 0.308 | Accuracy: 93%
[Step 23100] Past 100 steps: Average Loss 0.182 | Accuracy: 95%
[Step 23200] Past 100 steps: Average Loss 0.112 | Accuracy: 98%
[Step 23300] Past 100 steps: Average Loss 0.145 | Accuracy: 93%
[Step 23400] Past 100 steps: Average Loss 0.117 | Accuracy: 97%
[Step 23500] Past 100 steps: Average Los

[Step 34900] Past 100 steps: Average Loss 0.211 | Accuracy: 92%
[Step 35000] Past 100 steps: Average Loss 0.243 | Accuracy: 94%
[Step 35100] Past 100 steps: Average Loss 0.148 | Accuracy: 97%
[Step 35200] Past 100 steps: Average Loss 0.335 | Accuracy: 89%
[Step 35300] Past 100 steps: Average Loss 0.501 | Accuracy: 92%
[Step 35400] Past 100 steps: Average Loss 0.233 | Accuracy: 94%
[Step 35500] Past 100 steps: Average Loss 0.327 | Accuracy: 95%
[Step 35600] Past 100 steps: Average Loss 0.127 | Accuracy: 93%
[Step 35700] Past 100 steps: Average Loss 0.247 | Accuracy: 96%
[Step 35800] Past 100 steps: Average Loss 0.037 | Accuracy: 99%
[Step 35900] Past 100 steps: Average Loss 0.164 | Accuracy: 95%
[Step 36000] Past 100 steps: Average Loss 0.297 | Accuracy: 94%
[Step 36100] Past 100 steps: Average Loss 0.036 | Accuracy: 98%
[Step 36200] Past 100 steps: Average Loss 0.109 | Accuracy: 95%
[Step 36300] Past 100 steps: Average Loss 0.118 | Accuracy: 97%
[Step 36400] Past 100 steps: Average Los

[Step 47700] Past 100 steps: Average Loss 0.159 | Accuracy: 92%
[Step 47800] Past 100 steps: Average Loss 0.243 | Accuracy: 93%
[Step 47900] Past 100 steps: Average Loss 0.329 | Accuracy: 93%
[Step 48000] Past 100 steps: Average Loss 0.149 | Accuracy: 96%
[Step 48100] Past 100 steps: Average Loss 0.139 | Accuracy: 94%
[Step 48200] Past 100 steps: Average Loss 0.111 | Accuracy: 98%
[Step 48300] Past 100 steps: Average Loss 0.122 | Accuracy: 97%
[Step 48400] Past 100 steps: Average Loss 0.046 | Accuracy: 98%
[Step 48500] Past 100 steps: Average Loss 0.146 | Accuracy: 96%
[Step 48600] Past 100 steps: Average Loss 0.218 | Accuracy: 98%
[Step 48700] Past 100 steps: Average Loss 0.301 | Accuracy: 96%
[Step 48800] Past 100 steps: Average Loss 0.205 | Accuracy: 95%
[Step 48900] Past 100 steps: Average Loss 0.389 | Accuracy: 91%
[Step 49000] Past 100 steps: Average Loss 0.161 | Accuracy: 94%
[Step 49100] Past 100 steps: Average Loss 0.149 | Accuracy: 96%
[Step 49200] Past 100 steps: Average Los